In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import cv2
import tensorflow as tf
from tensorflow import keras

In [2]:
DATASET_PATH = '/content/drive/My Drive/University Of Stirling/Dissertation/retinal-rec/Datasets/APTOS2019'
TRAIN_PATH = DATASET_PATH + "/train_images/"
TEST_PATH = DATASET_PATH + "/test_images/"
TRAIN_PREP_PATH = DATASET_PATH + "/train_preprocessed/"
VALID_PREP_PATH = DATASET_PATH + "/valid_preprocessed/"
classes = [ "No DR", "Mild", "Moderate", "Severe", "Proliferative DR" ]
N_CLASSES = 5
IMG_SIZE = (512, 512)

In [3]:
from google.colab import drive
import os

drive.mount('/content/drive')

os.chdir(DATASET_PATH)
print("CWD:",os.getcwd())

Mounted at /content/drive
CWD: /content/drive/My Drive/University Of Stirling/Dissertation/retinal-rec/Datasets/APTOS2019


In preparation for the NN, it is necessary to create the correct directory structure.

In [ ]:
if not os.path.exists(DATASET_PATH + "/train"):
  os.mkdir(DATASET_PATH + "/train")
  os.mkdir(DATASET_PATH + "/train/0")
  os.mkdir(DATASET_PATH + "/train/1")
  os.mkdir(DATASET_PATH + "/train/2")
  os.mkdir(DATASET_PATH + "/train/3")
  os.mkdir(DATASET_PATH + "/train/4")

if not os.path.exists(DATASET_PATH + "/validation"):
  os.mkdir(DATASET_PATH + "/validation")
  os.mkdir(DATASET_PATH + "/validation/0")
  os.mkdir(DATASET_PATH + "/validation/1")
  os.mkdir(DATASET_PATH + "/validation/2")
  os.mkdir(DATASET_PATH + "/validation/3")
  os.mkdir(DATASET_PATH + "/validation/4")


In [ ]:
from sklearn.model_selection import train_test_split
import shutil

def read_dataset():
  df = pd.read_csv(DATASET_PATH + "/train.csv")
  
  # Train-test split
  train, valid = train_test_split(df, test_size=.3)

  for image in train.values:
    shutil.copyfile(TRAIN_PREP_PATH + image[0] + ".png", DATASET_PATH + "/train/" + str(image[1]) + "/" + image[0] + ".png")

  for image in valid.values:
    shutil.copyfile(TRAIN_PREP_PATH + image[0] + ".png", DATASET_PATH + "/validation/" + str(image[1]) + "/" + image[0] + ".png")
  
  return (train, valid)
  #train_labels = train_images["diagnosis"]
  #train_images = train_images["id_code"]

  #val_labels = val_images["diagnosis"]
  #val_images = val_images["id_code"]

  #return(train_images, train_labels, val_images, val_labels)

# Read labels


train, valid = read_dataset()

print(train.head(), valid.head())


           id_code  diagnosis
2114  949710bead24          0
2023  8e4a354e3da2          0
2550  b086c7cd3868          0
2783  c0202976c670          2
3520  f55e1d2a19e4          1            id_code  diagnosis
1885  84b88e8d3bca          0
41    032d7b0b4bf6          2
1158  51269b77d312          3
163   0c76fd494af6          2
516   24f271c87e73          0


In [4]:
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator


df = pd.read_csv(DATASET_PATH + "/train.csv")
  
# Train-test split
train, valid = train_test_split(df, test_size=.3)

# create a data generator
datagen = ImageDataGenerator()

# load and iterate training dataset
train_it = datagen.flow_from_directory(DATASET_PATH + "/train/", class_mode='binary', batch_size=64)
  
# load and iterate validation dataset
val_it = datagen.flow_from_directory(DATASET_PATH + "/validation/", class_mode='binary', batch_size=64)

Found 2563 images belonging to 5 classes.
Found 1099 images belonging to 5 classes.


In [8]:
from tensorflow.keras.utils import to_categorical

# AlexNet model structure definition
def AlexNet():
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation="softmax")])

model = AlexNet()

learning_rate = 0.001
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate = learning_rate), loss="categorical_crossentropy", metrics=['accuracy'])

# Print the shape of each layer
X = tf.random.uniform((1, 224, 224, 1))
for layer in AlexNet().layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

# One-Hot Encoding for labels
train_labels = to_categorical(train["diagnosis"])
val_labels = to_categorical(valid["diagnosis"])

# Normalize
#train_images = train_images/255.0
#val_images = val_images/255.0

# Training parameters
model.fit_generator(train_it, steps_per_epoch=4, validation_data=val_it, validation_steps=8, verbose=1)

Conv2D output shape:	 (1, 54, 54, 96)
MaxPooling2D output shape:	 (1, 26, 26, 96)
Conv2D output shape:	 (1, 26, 26, 256)
MaxPooling2D output shape:	 (1, 12, 12, 256)
Conv2D output shape:	 (1, 12, 12, 384)
Conv2D output shape:	 (1, 12, 12, 384)
Conv2D output shape:	 (1, 12, 12, 256)
MaxPooling2D output shape:	 (1, 5, 5, 256)
Flatten output shape:	 (1, 6400)
Dense output shape:	 (1, 4096)
Dropout output shape:	 (1, 4096)
Dense output shape:	 (1, 4096)
Dropout output shape:	 (1, 4096)
Dense output shape:	 (1, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


4/4 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.1077 

KeyboardInterrupt: ignored